### 8-4. 육아휴직 관련 법안에 대한 분석
 - import libraries
  * nltk
  * konlpy.corpus(kobill)
  * konlpy.tag(Okt)
  * matplotlib.pyplot
  * matplotlib(font_manager, rc)
  * platform
  * wordcloud(WordCloud)
 - No base image file.

In [ ]:
import nltk
from konlpy.corpus import kobill

In [ ]:
#1. 데이터 로딩
files_ko = kobill.fileids()
doc_ko = kobill.open('1809890.txt').read()  #Konlpy 내부문서 중 육아휴직 관련 텍스트.
doc_ko

In [ ]:
#2. Okt 분석기로 명사 분석
from konlpy.tag import Okt

okt = Okt()
tokens_ko = okt.nouns(doc_ko)

tokens_ko

In [ ]:
#3. 수집된 단어의 횟수와 고유한 횟수 확인

ko = nltk.Text(tokens_ko, name = '대한민국 국회 의안 제1809890호')
print(len(ko.tokens))  #수집된 단어의 횟수
print(len(set(ko.tokens))) #고유한 횟수

ko.vocab()     #각 단어의 빈도 분포

In [ ]:
#matplotlib로 한글 폰트 설정

import matplotlib.pyplot as plt
import platform

path = "C:/Windows/Fonts/malgun.ttf"

from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family = 'AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname = path).get_name()
    rc('font', family = font_name)
else:
    print('Unknown system...')
    
#그래프가 외부창에서 그려짐.
%matplotlib inline

In [ ]:
#4. 그래프로 빈도 표현
import matplotlib.pyplot as plt

plt.figure(figsize = (12, 6))
ko.plot(50)
plt.show()

In [ ]:
#5. stopwords 지정.

stopwords = ['.', ',', '(', ')', "'", '%', '-', 'X', ').', 'x', '의', '자', '에', '안', '번', '호',
            '을', '이', '다', '만', '로', '가', '를', '생', '략', '및', '곧', '것임', '느', '취']
ko = [each_word for each_word in ko if each_word not in stopwords] #ko에 있는 each_word 중 stopwords에 없는 것은 새 list인 ko에 넣어라.
ko

In [ ]:
#6. stopwords가 반영된 plot 다시 그려보기.
ko = nltk.Text(ko, name = '대한민국 국회 의안 제1809890호')
print(ko)
print(ko.vocab())
plt.figure(figsize = (12, 6))
ko.plot(50)    #출현 빈도 상위 50개 토큰
plt.show()

In [ ]:
#7. 특정 단어가 몇 번 언급되었는지 확인
ko.count('지방공무원법')

In [ ]:
#8. 원하는 단어의 문서 내 개략적인 위치 + 분량
plt.figure(figsize = (12, 6))
ko.dispersion_plot(['육아휴직', '초등학교', '공무원'])

In [ ]:
#import nltk
#nltk.download('stopwords')

#9. 원하는 단어의 주변부 단어까지 같이 확인.
print(ko.concordance('초등학교'))
print()

#연어(collocation) 파악 가능
print(ko.collocations())

In [ ]:
#10. wordcloud todtjd
from wordcloud import WordCloud

data = ko.vocab().most_common(100)   #최빈 100개 단어
print(data)
wordcloud = WordCloud(font_path = 'C:/Windows/Fonts/malgun.ttf',
                     relative_scaling = 0.2,
                     background_color = 'black',
                     ).generate_from_frequencies(dict(data))

plt.figure(figsize = (10, 10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

### 8-5. Naive Bayes Classifier의 이해(영문)
- import libraries
  * nltk.tokenize(word_tokenize)
  * nltk

In [ ]:
#심플한 분류기: Naive Bayes
#지도학습의 한 종류.
#두 사건을 서로 독립이라고 가정하고, 각각의 조건부확률을 활용.

In [ ]:
from nltk.tokenize import word_tokenize
import nltk

#1. 연습용 데이터 4개 문장(문장, 긍/부정 label)
train = [('i like you', 'pos'),
         ('i hate you', 'neg'),
         ('you like me', 'neg'),
         ('i like her', 'pos')]  #i가 등장할 때 like는 긍정, like 자체만으로는 긍/부정 판단 불가.

In [ ]:
nltk.download('punkt')

all_words = set(word.lower() for sentence in train for word in word_tokenize(sentence[0]))
all_words #train 문장에서 사용된 전체 단어를 찾는다.

#all_words를 말뭉치라고 하자.

In [ ]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in train]
t  #말뭉치(all_words)를 기준으로 train 문장에 속한 단어인지 아닌지를 기록.
   #True = 이 문장에 있는 단어, False = 이 문장에 없는 단어

In [ ]:
# 분류기 실행
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()
# Train 문장에 붙은 긍/부정 태그를 이용하여 분류한 결과
# hate라는 단어가 없을 때(=False일 때) 긍정일 비율이 1.7:1
# like는 총 3번 사용되었고, 그 중 2개의 문장이 긍정의 의미였으므로 like가 있을 때(=True일 때) 긍정일 확률이 1.7:1

In [ ]:
#테스트 문장으로 나이브-베이즈 분류기 실행
test_sentence = 'i like MeRui'
test_sent_feature = {word.lower():
                    (word in word_tokenize(test_sentence.lower())) for word in all_words}
test_sent_feature

In [ ]:
classifier.classify(test_sent_feature)    #테스트 문장 분류 결과: 긍정 

### 8-6. Naive Bayes Classifier의 이해(한글)
- import libraries
  * konlpy.tag(Okt)
  * nltk.tokenize(word_tokenize)
  * nltk

In [ ]:
from konlpy.tag import Okt
pos_tagger = Okt()

#영어와 달리, 한글은 형태소 분석을 반드시 해야 한다.
#하지 않으면 어떻게 되는지 먼저 보자.

In [ ]:
#train 문장 생성
train = [('메리가 좋아', 'pos'),
         ('고양이도 좋아', 'pos'),
         ('난 수업이 지루해', 'neg'),
         ('메리는 예쁜 고양이야', 'pos'),
         ('난 마치고 메리랑 놀 거야', 'pos')]

In [ ]:
#말뭉치 만들기
from nltk.tokenize import word_tokenize
import nltk

all_words = set(word.lower() for sentence in train for word in word_tokenize(sentence[0]))
all_words  #'고양이도', '고양이야'가 다른 단어로 잡힌다.

In [ ]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in train]
t  #각 단어가 train문장에 속했는지 아닌지 확인

In [ ]:
#분류
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

In [ ]:
#테스트 문장
test_sentence = '난 수업이 마치면 메리랑 놀 거야'

In [ ]:
#문장만 보면 긍정적 결과가 나와야 할 것 같다.
test_sent_feature = {word.lower(): word in word_tokenize(test_sentence.lower())
                    for word in all_words}
test_sent_feature

In [ ]:
#그러나 실제로 적용하면, 'neg'를 출력한다.
classifier.classify(test_sent_feature)

#결국, 한글에서는 형태소 분석이 필요한 것이다.

In [ ]:
#아래 함수처럼 tag를 붙여주는 것이 유리함.
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm = True, stem = True)]

In [ ]:
#위의 tokenize 함수를 사용해서 train 문장 분석.
train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs

In [ ]:
#전체 말뭉치 생성
tokens = [t for d in train_docs for t in d[0]]
tokens

In [ ]:
#말뭉치에 있는 단어인지 아닌지를 구분하는 함수를 만들고 train 문장에 적용.
#조사, 명사의 구분이 잘 되어있는 것이 특징.
def term_exists(doc):
    return {word: (word in set(doc)) for word in tokens}

train_xy = [(term_exists(d), c) for d, c in train_docs]
train_xy

In [ ]:
#분류기 동작시키기!
classifier = nltk.NaiveBayesClassifier.train(train_xy)

In [ ]:
#테스트 문장으로 다시 테스트
test_sentence = [("난 수업이 마치면 메리랑 놀 거야")]

In [ ]:
#형태소분석 실시
test_docs = pos_tagger.pos(test_sentence[0])
test_docs

In [ ]:
test_sent_feature = {word: (word in tokens) for word in test_docs}
test_sent_feature

In [ ]:
# 테스트 문장 분류 결과: pos
classifier.classify(test_sent_feature)